In [1]:
import pandas as pd

In [2]:
fires_origin = pd.read_csv(
    "forestfires.csv", header=0, index_col=False) 
d = {
    'X': fires_origin['X'].values, 
    'Y': fires_origin['Y'].values, 
    'temp': fires_origin['temp'].values,
    'RH': fires_origin['RH'].values,
    'wind': fires_origin['wind'].values,
    'rain': fires_origin['rain'].values,
    'area': fires_origin['area'].values
}
fires = pd.DataFrame(d)
print(fires.head())

   X  Y  temp  RH  wind  rain  area
0  7  5   8.2  51   6.7   0.0   0.0
1  7  4  18.0  33   0.9   0.0   0.0
2  7  4  14.6  33   1.3   0.0   0.0
3  8  6   8.3  97   4.0   0.2   0.0
4  8  6  11.4  99   1.8   0.0   0.0


In [3]:
import statistics as stat
maximum = max(fires['temp'])
minimum = min(fires['temp'])
median = stat.median(fires['temp'])
diff = round((maximum - median)/4)
max_RH = max(fires['RH'])
min_RH = min(fires['RH'])
median_RH = stat.median(fires['RH'])
max_wind= max(fires['wind'])
min_wind= min(fires['wind'])
median_wind= stat.median(fires['wind'])

In [4]:
tp = fires['temp']
rh = fires['RH']
w = fires['wind']
serious = fires['temp'].copy()
for i, val in enumerate(fires['temp']):
    #print(val, rh[i], serious[i])
    if (val >= (median + diff) and rh[i] <= 30 and w[i] >= 3.0):
        serious.loc[i] = 4
    else:
        if (val >= median + diff):
            serious.loc[i] = 3
        else: 
            if (val >= (median - diff) and rh[i] <= 50 and w[i] >= 4.0):
                serious.loc[i] = 3
            else: 
                if (val >= median - 2*diff and rh[i] <= 50 and w[i] >= 5.0):
                    serious.loc[i] = 2
                else:
                    if (val >= median - 2*diff):
                        serious.loc[i] = 1
                    else:
                        serious.loc[i] = 0

In [5]:
print(serious[1:20])

1     1.0
2     1.0
3     0.0
4     0.0
5     3.0
6     4.0
7     0.0
8     0.0
9     3.0
10    1.0
11    3.0
12    1.0
13    1.0
14    4.0
15    3.0
16    2.0
17    3.0
18    1.0
19    0.0
Name: temp, dtype: float64


In [6]:
x = pd.DataFrame(
    fires.values[:,0:11], 
    columns =['X','Y','temp', 'RH', 'wind', 'rain', 'area']
)
y = pd.DataFrame(serious.values, columns =['SERIOUS'])

In [7]:
print(x.shape)
print(y.shape)

(517, 7)
(517, 1)


In [8]:
print(x[1:5])
print(y[1:5])

     X    Y  temp    RH  wind  rain  area
1  7.0  4.0  18.0  33.0   0.9   0.0   0.0
2  7.0  4.0  14.6  33.0   1.3   0.0   0.0
3  8.0  6.0   8.3  97.0   4.0   0.2   0.0
4  8.0  6.0  11.4  99.0   1.8   0.0   0.0
   SERIOUS
1      1.0
2      1.0
3      0.0
4      0.0


In [9]:
# Optymalizacja modelu

In [10]:
# Grid Search - jedna zmienna
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
max_depth = np.array(
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
model = RandomForestClassifier(n_estimators=2)
grid = GridSearchCV(estimator=model, param_grid=dict(max_depth=max_depth))
grid.fit(x, y.values.ravel())

GridSearchCV(estimator=RandomForestClassifier(n_estimators=2),
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25])})

In [11]:
print(grid)

GridSearchCV(estimator=RandomForestClassifier(n_estimators=2),
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25])})


In [12]:
print(grid.cv_results_['params'])

[{'max_depth': 1}, {'max_depth': 2}, {'max_depth': 3}, {'max_depth': 4}, {'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}, {'max_depth': 20}, {'max_depth': 21}, {'max_depth': 22}, {'max_depth': 23}, {'max_depth': 24}, {'max_depth': 25}]


In [13]:
print(grid.cv_results_['mean_test_score'])

[0.52634429 0.66910007 0.66740105 0.69245706 0.8492345  0.79473488
 0.85505975 0.85494772 0.82393577 0.77361837 0.82401046 0.78534354
 0.81422704 0.81441374 0.81056759 0.84919716 0.8122106  0.81639283
 0.8317401  0.80485437 0.82421583 0.84921583 0.78918969 0.82942494
 0.85296863]


In [14]:
print(grid.best_params_)

{'max_depth': 7}


In [15]:
# Grid Search - dwie zmienne
max_depth = np.array(
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
n_estimators = np.array([2,3,4,5,6,7,8,9,10])
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, 
                    param_grid=dict(
                        n_estimators=n_estimators, 
                        max_depth=max_depth))
grid.fit(x, y.values.ravel())
print(grid.cv_results_['params'])

[{'max_depth': 1, 'n_estimators': 2}, {'max_depth': 1, 'n_estimators': 3}, {'max_depth': 1, 'n_estimators': 4}, {'max_depth': 1, 'n_estimators': 5}, {'max_depth': 1, 'n_estimators': 6}, {'max_depth': 1, 'n_estimators': 7}, {'max_depth': 1, 'n_estimators': 8}, {'max_depth': 1, 'n_estimators': 9}, {'max_depth': 1, 'n_estimators': 10}, {'max_depth': 2, 'n_estimators': 2}, {'max_depth': 2, 'n_estimators': 3}, {'max_depth': 2, 'n_estimators': 4}, {'max_depth': 2, 'n_estimators': 5}, {'max_depth': 2, 'n_estimators': 6}, {'max_depth': 2, 'n_estimators': 7}, {'max_depth': 2, 'n_estimators': 8}, {'max_depth': 2, 'n_estimators': 9}, {'max_depth': 2, 'n_estimators': 10}, {'max_depth': 3, 'n_estimators': 2}, {'max_depth': 3, 'n_estimators': 3}, {'max_depth': 3, 'n_estimators': 4}, {'max_depth': 3, 'n_estimators': 5}, {'max_depth': 3, 'n_estimators': 6}, {'max_depth': 3, 'n_estimators': 7}, {'max_depth': 3, 'n_estimators': 8}, {'max_depth': 3, 'n_estimators': 9}, {'max_depth': 3, 'n_estimators': 10

In [16]:
print(model)

RandomForestClassifier()


In [17]:
print(grid.cv_results_['mean_test_score'])

[0.51256535 0.60358476 0.63086258 0.60776699 0.56667289 0.58816281
 0.58627707 0.64802091 0.65752427 0.59016057 0.67498133 0.71926811
 0.73722928 0.74255041 0.75418223 0.74858103 0.75070948 0.77948096
 0.71000747 0.75057879 0.75464899 0.77005228 0.79307319 0.77197535
 0.845295   0.85115758 0.81833458 0.78334578 0.69667662 0.83930172
 0.83754668 0.84352128 0.89171023 0.86463779 0.8568708  0.86656087
 0.82382375 0.83360717 0.88988051 0.82022031 0.88000373 0.89762883
 0.89169156 0.89930919 0.88780807 0.79124347 0.8801531  0.90903659
 0.88392457 0.91489918 0.88211352 0.92270351 0.8896938  0.90724421
 0.85084018 0.89736744 0.93033981 0.91497386 0.93820015 0.91480583
 0.92658701 0.93224421 0.94195295 0.81413368 0.88967513 0.89940254
 0.92055639 0.94777819 0.91095967 0.94202763 0.95160568 0.93226288
 0.86460045 0.90916729 0.91874533 0.91491785 0.91484317 0.93420463
 0.93030246 0.94585512 0.94581777 0.82819268 0.85681479 0.8917289
 0.9321882  0.93609037 0.9168596  0.93043316 0.93039582 0.93220

In [18]:
print(grid.best_params_)

{'max_depth': 25, 'n_estimators': 9}


In [19]:
criterion = np.array(['gini','entropy'])
max_depth = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25])
n_estimators = np.array([2,3,4,5,6,7,8,9,10])
model = RandomForestClassifier()
grid = GridSearchCV(estimator=model, param_grid=dict(
    n_estimators=n_estimators, max_depth=max_depth, criterion=criterion))
grid.fit(x, y.values.ravel())
print(grid.cv_results_['params'])

[{'criterion': 'gini', 'max_depth': 1, 'n_estimators': 2}, {'criterion': 'gini', 'max_depth': 1, 'n_estimators': 3}, {'criterion': 'gini', 'max_depth': 1, 'n_estimators': 4}, {'criterion': 'gini', 'max_depth': 1, 'n_estimators': 5}, {'criterion': 'gini', 'max_depth': 1, 'n_estimators': 6}, {'criterion': 'gini', 'max_depth': 1, 'n_estimators': 7}, {'criterion': 'gini', 'max_depth': 1, 'n_estimators': 8}, {'criterion': 'gini', 'max_depth': 1, 'n_estimators': 9}, {'criterion': 'gini', 'max_depth': 1, 'n_estimators': 10}, {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 2}, {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 3}, {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 4}, {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 5}, {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 6}, {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 7}, {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 8}, {'criterion': 'gini', 'max_depth': 2, 'n_estimators': 9}, {'criterion'

In [20]:
print(grid.cv_results_['mean_test_score'])

[0.52791262 0.5764003  0.55532114 0.5610717  0.57068708 0.68069455
 0.60393951 0.60541449 0.69820762 0.69251307 0.71372293 0.72345034
 0.68302838 0.71368559 0.70997013 0.69798357 0.78330844 0.81049291
 0.7893764  0.76387229 0.76809186 0.78717326 0.74843167 0.82976102
 0.79324122 0.79712472 0.84333458 0.81222928 0.84148618 0.84729276
 0.85888723 0.84143017 0.85093353 0.8451643  0.83573562 0.86086632
 0.7739171  0.84370799 0.89359597 0.89929052 0.85289395 0.86661688
 0.88584765 0.92458925 0.90339806 0.83560493 0.84161688 0.8510829
 0.92089246 0.91306945 0.91882001 0.89553771 0.93226288 0.91870799
 0.82401046 0.89940254 0.92462659 0.89546303 0.90321135 0.92647498
 0.92261016 0.91497386 0.92460792 0.77964899 0.88000373 0.91678491
 0.93235624 0.95162435 0.94779686 0.94010456 0.93614638 0.95364078
 0.79490291 0.87225541 0.91682226 0.918764   0.92270351 0.92660568
 0.91292009 0.93622106 0.93616505 0.84159821 0.8896938  0.93231889
 0.94585512 0.93812547 0.92070575 0.93991785 0.96713966 0.95552

In [21]:
print(grid.best_params_)

{'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 9}


In [22]:
# Zadanie
# Proszę pobrać dowolny zbiór danych ze strony https://archive.ics.uci.edu/ml/index.php
# Następnie proszę zoptymalizować las metodą Grid Search (trzy parametry modelu mają zostać optymalizowane).
# Jakie wartości parametrów są najlepsze i dlaczego?